In [1]:
!pip install nltk
!python -m spacy download en
!pip install summa
!pip install spacy
!pip install markovify
!pip install -m spacy download en
!pip install pymorphy2

from math import sqrt
import copy

import pandas as pd
import numpy as np
import markovify
import nltk
import spacy
import re
import nltk
import warnings
import json
import pymorphy2

import gensim
import gensim.downloader as download_api
from gensim.models import Word2Vec

from pymystem3 import Mystem
from string import punctuation
from pprint import pprint
from summa.keywords import keywords

from nltk.corpus import gutenberg
from nltk import download, sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import RegexpTokenizer

nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download("punkt")
nltk.download('stopwords')
warnings.filterwarnings('ignore')
russian_model = download_api.load('word2vec-ruscorpora-300')

     |████████████████████████████████| 12.0 MB 8.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -m
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punk

In [59]:
emotion_list = ['отвращение', 'страх', 'злость', 'стыд', 'радость', 'горе', 'волнение', 'грусть']
useful_pos_list = ['INFN', 'NOUN', 'ADJF', 'ADJS', 'VERB', 'ADVB']

import copy

def tag(word):
  morph = pymorphy2.MorphAnalyzer(lang='ru')

  w = morph.parse(word)[0].tag.POS
  tg = copy.deepcopy(w)
  if w == 'ADJF' or w == 'ADJS':
    w = 'ADJ'
  elif w == 'ADVB':
    w = 'ADV'
  elif w == 'INFN':
    w = 'VERB'
  tagged = str(word) + '_' + str(w)

  return tagged

#please pass text as copy of python string
def preprocess_phrase(phrase):
  morph = pymorphy2.MorphAnalyzer(lang='ru')

  phrase = phrase.lower()
  phrase = re.sub(r'[^\w\s]','', phrase) 

  phrase_array = phrase.split()
  useful_phrases = []
  for i in phrase_array:
    cur_word = morph.parse(i)[0].normalized
    if cur_word.tag.POS in useful_pos_list:
      useful_phrases.append(cur_word.word)
  return useful_phrases

def get_emotion_array(phrase):

  contest = preprocess_phrase(phrase)

  if len(contest) == 0:
    print("Недостаточно слов для анализа")
    return []

  emotion_rank = []
  for emotion in emotion_list:
    cur_vec = []
    counter = 0
    for word in contest:
      try:
        score = russian_model.similarity(tag(word), tag(emotion))
        cur_vec.append(score ** 2.5)
        counter+=1
      except:
        cur_vec.append(0.0)
    
    if counter == 0:
      print("Недостаточно слов для анализа")
      return []
    cur_vec.sort(reverse=True)
    cur_vec = cur_vec[:max(3, len(contest))]
    cur_score = 0.0
    for scr in cur_vec:
      cur_score+=scr
    cur_score/=len(cur_vec)
    emotion_rank.append(cur_score)
  max_score = 0.0
  for scr in emotion_rank:
    max_score = max(scr, max_score)
  for i in range(len(emotion_rank)):
    emotion_rank[i] = emotion_rank[i]/max_score
  ans = []
  for i in range(len(emotion_list)):
    ans.append((emotion_list[i], emotion_rank[i]))
  return ans


In [60]:
print(get_emotion_array('Я безумно хочу пиццу'))

[('отвращение', 0.8819056097826328), ('страх', 0.7709850468016959), ('злость', 0.7930077574873069), ('стыд', 0.6676257774997041), ('радость', 1.0), ('горе', 0.7975960158205192), ('волнение', 0.41319201790815047), ('грусть', 0.5263288848408975)]


In [61]:
print(get_emotion_array('Как же я нелюблю машинное обучение'))

[('отвращение', 0.3984809320371697), ('страх', 0.7204672073307211), ('злость', 0.7860605679274367), ('стыд', 0.13277793975425423), ('радость', 0.47781310812380073), ('горе', 0.08036397935808925), ('волнение', 1.0), ('грусть', 0.06592442946789252)]


In [62]:
print(get_emotion_array('Осенний пейзаж заставил меня пустить скупую слезу'))

[('отвращение', 0.28274899145471855), ('страх', 0.35783146060261917), ('злость', 0.5283499105151671), ('стыд', 0.5158890825492591), ('радость', 0.7181388836330391), ('горе', 0.7495103231527452), ('волнение', 0.414300243825735), ('грусть', 1.0)]
